In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [3]:
df = describe_dataset("./CSV_dataset/train.csv")
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2
df.tail(3)

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
1055,0,0.240020,0.473348,-0.035444,0.999667,0.338099,0.483100,-0.177646,0.998425,0.358254,...,0.227517,0.617147,0.927214,0.688592,-0.048634,0.890224,0.897454,0.658981,0.185193,0.658671
1056,0,0.272779,0.617412,-0.585275,0.999738,0.415411,0.552398,-0.684503,0.999507,0.318591,...,1.093460,0.506005,0.770820,0.667592,0.660936,0.721354,0.733822,0.651491,1.065337,0.466834
1057,0,0.188976,0.665301,0.011176,0.999974,0.254258,0.650633,-0.154442,0.999830,0.282009,...,0.110239,0.777685,0.707999,0.824241,-0.268242,0.955986,0.700774,0.808693,0.059137,0.817536


In [4]:
# Extract features and class
X = df.drop("label", axis=1)
y = df["label"].astype("int")

In [5]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(3)

827    2
892    0
205    0
Name: label, dtype: int32

In [7]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []

for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=[0, 1, 2])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=[0, 1, 2])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=[0, 1, 2])
    cm = confusion_matrix(y_test, model_results, labels=[0, 1, 2])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))

In [8]:
# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)

pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,KNN,"[0.987, 1.0, 0.982]",0.990566,"[0.987, 0.988, 1.0]","[0.987, 0.994, 0.991]","[[75, 0, 1], [1, 79, 0], [0, 0, 56]]"
1,SVC,"[0.962, 1.0, 1.0]",0.985849,"[1.0, 0.988, 0.964]","[0.981, 0.994, 0.982]","[[76, 0, 0], [1, 79, 0], [2, 0, 54]]"
2,LR,"[0.962, 1.0, 0.982]",0.981132,"[0.987, 0.988, 0.964]","[0.974, 0.994, 0.973]","[[75, 0, 1], [1, 79, 0], [2, 0, 54]]"
3,SGDC,"[0.962, 1.0, 0.982]",0.981132,"[0.987, 0.988, 0.964]","[0.974, 0.994, 0.973]","[[75, 0, 1], [1, 79, 0], [2, 0, 54]]"
4,RF,"[0.95, 1.0, 1.0]",0.981132,"[1.0, 0.988, 0.946]","[0.974, 0.994, 0.972]","[[76, 0, 0], [1, 79, 0], [3, 0, 53]]"
5,DTC,"[0.936, 0.975, 0.963]",0.957547,"[0.961, 0.975, 0.929]","[0.948, 0.975, 0.945]","[[73, 2, 1], [1, 78, 1], [4, 0, 52]]"
6,NB,"[0.526, 0.778, 0.677]",0.655660,"[0.539, 0.7, 0.75]","[0.532, 0.737, 0.712]","[[41, 15, 20], [24, 56, 0], [13, 1, 42]]"


In [9]:
test_df = describe_dataset("./CSV_dataset/test.csv")
test_df = test_df.sample(frac=1).reset_index(drop=True)

test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"].astype("int")

test_x = pd.DataFrame(sc.transform(test_x))

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

In [10]:
testset_final_results = []

for name, model in models.items():
    # Evaluate model
    model_results = model.predict(test_x)

    p_score = precision_score(test_y, model_results, average=None, labels=[0, 1, 2])
    a_score = accuracy_score(test_y, model_results)
    r_score = recall_score(test_y, model_results, average=None, labels=[0, 1, 2])
    f1_score_result = f1_score(test_y, model_results, average=None, labels=[0, 1, 2])
    cm = confusion_matrix(test_y, model_results, labels=[0, 1, 2])
    testset_final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm ))


testset_final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(testset_final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,RF,"[0.945, 0.988, 1.0]",0.976134,"[0.993, 0.988, 0.939]","[0.968, 0.988, 0.969]","[[138, 1, 0], [2, 163, 0], [6, 1, 108]]"
1,LR,"[0.93, 0.97, 0.982]",0.959427,"[0.957, 0.97, 0.948]","[0.943, 0.97, 0.965]","[[133, 4, 2], [5, 160, 0], [5, 1, 109]]"
2,SVC,"[0.919, 0.987, 0.973]",0.959427,"[0.978, 0.952, 0.948]","[0.948, 0.969, 0.96]","[[136, 1, 2], [7, 157, 1], [5, 1, 109]]"
3,SGDC,"[0.917, 0.975, 0.964]",0.952267,"[0.95, 0.964, 0.939]","[0.933, 0.97, 0.952]","[[132, 3, 4], [6, 159, 0], [6, 1, 108]]"
4,DTC,"[0.894, 0.97, 0.927]",0.933174,"[0.914, 0.988, 0.878]","[0.904, 0.979, 0.902]","[[127, 4, 8], [2, 163, 0], [13, 1, 101]]"
5,KNN,"[0.891, 0.951, 0.907]",0.918854,"[0.885, 0.939, 0.93]","[0.888, 0.945, 0.918]","[[123, 7, 9], [8, 155, 2], [7, 1, 107]]"
6,NB,"[0.493, 0.831, 0.642]",0.646778,"[0.54, 0.655, 0.765]","[0.515, 0.732, 0.698]","[[75, 21, 43], [51, 108, 6], [26, 1, 88]]"


In [11]:
with open("C:/Users/Alrowad/Exercise-correction/plank_model/models/all_sklearn.pkl", "wb") as f:
    pickle.dump(models, f)

In [13]:
with open("C:/Users/Alrowad/Exercise-correction/plank_model/models/RF_model.pkl", "wb") as f:
    pickle.dump(models["RF"], f)

In [14]:
with open("C:/Users/Alrowad/Exercise-correction/plank_model/models/SVC_model.pkl", "wb") as f:
    pickle.dump(models["SVC"], f)

In [15]:
with open("C:/Users/Alrowad/Exercise-correction/plank_model/models/LR_model.pkl", "wb") as f:
    pickle.dump(models["LR"], f)

In [16]:
# Dump input scaler
with open("C:/Users/Alrowad/Exercise-correction/plank_model/models/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)